In [1]:
import pickle

In [2]:
file_names = ['url_list_women.p', 'url_list_women_sale.p', 
              'url_list_men.p', 'url_list_men_sale.p']

In [3]:
item_url_list = []

In [4]:
for file_name in file_names:
    f = open(file_name, 'rb')
    item_url_list = item_url_list + pickle.load(f)
    f.close()

In [5]:
len(item_url_list)

8528

In [6]:
item_url_list[:10]

['http://www.hm.com/us/product/65274?article=65274-B',
 'http://www.hm.com/us/product/65274?article=65274-E',
 'http://www.hm.com/us/product/67562?article=67562-D',
 'http://www.hm.com/us/product/54630?article=54630-F',
 'http://www.hm.com/us/product/66996?article=66996-B',
 'http://www.hm.com/us/product/22590?article=22590-V',
 'http://www.hm.com/us/product/67571?article=67571-A',
 'http://www.hm.com/us/product/70360?article=70360-A',
 'http://www.hm.com/us/product/70480?article=70480-C',
 'http://www.hm.com/us/product/64155?article=64155-B']

In [7]:
import urllib
def return_item_page(url_item):
    handle = urllib.urlopen(url_item)
    html_gunk =  handle.read()
    handle.close()
    return str(html_gunk)

In [8]:
def get_metricCategoryID(item_page):
    idx_start = item_page.find("metricCategoryID") + len('metricCategoryID":"')
    idx_end = idx_start + item_page[idx_start:].find('"')
    
    return item_page[idx_start:idx_end]

def check_metricCategoryID(metricCategoryID):
    category_list = ['TOPS', 'BOTTOMS', 'SHIRT', 'VESTS' 'BLOUSES', 'DRESSES', 'JUMPSUITS', 'JEANS', 'PANTS', 'TROUSERS', 'CARDIGANS', 'SWEATERS', 'JUMPERS', 'HOODIES', 
               'SWEATSHIRTS', 'SHORTS', 'SKIRTS', 'JACKETS', 'COATS', 'BLAZERS', 'SUITS']
    for category in category_list:
        if metricCategoryID.find(category) != -1:
            return True
    return False

def check_category(item_page):
    return check_metricCategoryID(get_metricCategoryID(item_page))

In [9]:
def get_info(url_item):
    # url_item = "http://www.hm.com/us/product/72818?article=72818-A"
    item_page = return_item_page(url_item)
    
    metricCategoryID = get_metricCategoryID(item_page)
    
    # This 'if' statement is for speeding up 'get_info'
    if check_metricCategoryID(metricCategoryID) == False:
        return -1 # Failed
    
    parser = HnM_ProductPage_Parser()
    parser.feed(item_page)

    item_serial = url_item[url_item.find("article=")+len("article="):]

    return {'serial':item_serial, 
            'name':parser.get_name(), 
            'color':parser.get_color(), 
            'who':parser.get_who(), 
            'image': { 
                'model':parser.get_image_url_model(), 
                'item':parser.get_image_url_item()
                }, 
            #'title':parser.get_title(), 
            #'title breaks':parser.get_title_breaks(), 
            'url':url_item, 
            'metricCategoryID':metricCategoryID}

In [10]:
import urllib
from HTMLParser import HTMLParser

class HnM_ProductPage_Parser(HTMLParser):
    in_head_tag = False
    in_title_tag = False
    image_url_model = ""
    image_url_item = ""
    name = ""
    color = ""
    who = ""
    title = ""
    title_breaks = 0
    
    def handle_starttag(self, tag, attrs):
        if tag == "head":
            self.in_head_tag = True
        if tag == "title":
            self.in_title_tag = True
        if tag == "a":
            for attr in attrs:
                if attr[0] == "class" and (attr[1] == "FASHION_FRONT" or attr[1] == "STUDIO_FRONT"):
                    for attr2 in attrs:
                        if attr2[0] == "href":
                            self.image_url_model = "http:" + attr2[1]
                
                if attr[0] == "class" and attr[1] == "STILL_LIFE_FRONT":
                    for attr2 in attrs:
                        if attr2[0] == "href":
                            self.image_url_item = "http:" + attr2[1]
        
    def handle_endtag(self, tag):
        if tag == "head":
            self.in_head_tag = False
        if tag == "title":
            self.in_title_tag = False
            data_list = self.title.split(' | ')
            if len(data_list) == 4:
                self.name =  data_list[0]
                self.color = data_list[1]
                self.who = data_list[-2]
            if len(data_list) == 3:
                self.name =  data_list[0]
                self.who = data_list[-2]
                
    def handle_data(self, data):
        if self.in_head_tag and self.in_title_tag: # handling an unknown error 
            if self.title == "":
                self.title = self.title + data
            else:
                self.title = self.title + '&M' + data
            self.title_breaks = self.title_breaks + 1
        
    def get_image_url_model(self):
        return self.image_url_model
    
    def get_image_url_item(self):
        return self.image_url_item
    
    def get_name(self):
        return self.name
    
    def get_color(self):
        return self.color
    
    def get_who(self):
        return self.who
    
    def get_title(self):
        return self.title
    
    def get_title_breaks(self):
        return self.title_breaks

# Get the information of one item

In [11]:
item_info = get_info('http://www.hm.com/us/product/65274?article=65274-B') # T-shirt

In [12]:
item_info

{'color': 'White',
 'image': {'item': 'http://lp.hm.com/hmprod?set=key[source],value[/model/2017/E00 0478110 001 77 1688.jpg]&set=key[rotate],value[]&set=key[width],value[]&set=key[height],value[]&set=key[x],value[]&set=key[y],value[]&set=key[type],value[STILL_LIFE_FRONT]&set=key[hmver],value[2]&call=url[file:/product/large]',
  'model': 'http://lp.hm.com/hmprod?set=key[source],value[/environment/2017/8GZ_0158_011R.jpg]&set=key[rotate],value[0]&set=key[width],value[3927]&set=key[height],value[4591]&set=key[x],value[0]&set=key[y],value[-1]&set=key[type],value[FASHION_FRONT]&set=key[hmver],value[0]&call=url[file:/product/large]'},
 'metricCategoryID': 'LADIES_TOPS_SHORT_SLEEVED_W112',
 'name': 'Short-sleeved Top',
 'serial': '65274-B',
 'url': 'http://www.hm.com/us/product/65274?article=65274-B',
 'who': 'Women'}

In [13]:
item_info = get_info('http://www.hm.com/us/product/64141?article=64141-E') # shoes

In [14]:
item_info

-1

# Get the information of all items

In [15]:
items_info = []

In [16]:
len(item_url_list)

8528

In [17]:
count_loop = 0

In [18]:
for i in range(count_loop, len(item_url_list)):
    info = get_info(item_url_list[i]) 
    if info != -1:
        items_info.append(info)
    count_loop = count_loop + 1
    if count_loop % 100 == 1:
        print "count loops:" , count_loop, "===================================="
print "total count loops:" , count_loop

count loops: 1 ====================================
count loops: 101 ====================================
count loops: 201 ====================================
count loops: 301 ====================================
count loops: 401 ====================================
count loops: 501 ====================================
count loops: 601 ====================================
count loops: 701 ====================================
count loops: 801 ====================================
count loops: 901 ====================================
count loops: 1001 ====================================
count loops: 1101 ====================================
count loops: 1201 ====================================
count loops: 1301 ====================================
count loops: 1401 ====================================
count loops: 1501 ====================================
count loops: 1601 ====================================
count loops: 1701 ====================================
count loops: 1801 ====

In [19]:
print count_loop, len(items_info)

8528 5607


* 8528: Total women's and men's items
* 5607: Totat women's and men's items that are tops or bottoms

In [20]:
count_loop == len(item_url_list)

True

### Check the data

In [21]:
import random

In [22]:
item_info = random.choice(items_info)

In [24]:
item_info

{'color': 'Red/floral',
 'image': {'item': 'http://lp.hm.com/hmprod?set=key[source],value[/model/2017/E00 0502168 006 24 3626.jpg]&set=key[rotate],value[]&set=key[width],value[]&set=key[height],value[]&set=key[x],value[]&set=key[y],value[]&set=key[type],value[STILL_LIFE_FRONT]&set=key[hmver],value[1]&call=url[file:/product/large]',
  'model': 'http://lp.hm.com/hmprod?set=key[source],value[/environment/2017/8TZ_0332_001R.jpg]&set=key[rotate],value[0]&set=key[width],value[3776]&set=key[height],value[4416]&set=key[x],value[0]&set=key[y],value[-1]&set=key[type],value[FASHION_FRONT]&set=key[hmver],value[0]&call=url[file:/product/large]'},
 'metricCategoryID': 'LADIES_DIVIDED_DRESSES_JUMPSUITS_W041',
 'name': 'Off-the-shoulder Dress',
 'serial': '69333-A',
 'url': 'http://www.hm.com/us/product/69333?article=69333-A',
 'who': 'Women'}

A correct result

In [25]:
random.choice(items_info)

{'color': 'Khaki',
 'image': {'item': 'http://lp.hm.com/hmprod?set=key[source],value[/model/2017/E00 0259609 013 18 2083.jpg]&set=key[rotate],value[]&set=key[width],value[]&set=key[height],value[]&set=key[x],value[]&set=key[y],value[]&set=key[type],value[STILL_LIFE_FRONT]&set=key[hmver],value[1]&call=url[file:/product/large]',
  'model': 'http://lp.hm.com/hmprod?set=key[source],value[/environment/2017/8LZ_0031_016R.jpg]&set=key[rotate],value[0]&set=key[width],value[3998]&set=key[height],value[4674]&set=key[x],value[0]&set=key[y],value[-1]&set=key[type],value[FASHION_FRONT]&set=key[hmver],value[0]&call=url[file:/product/large]'},
 'metricCategoryID': 'MEN_CASUAL_SHORTS_M083',
 'name': 'Cargo Shorts',
 'serial': '56824-G',
 'url': 'http://www.hm.com/us/product/56824?article=56824-G',
 'who': 'Men'}

A correct result

In [26]:
random.choice(items_info)

{'color': 'Black',
 'image': {'item': 'http://lp.hm.com/hmprod?set=key[source],value[/model/2016/D00 0443566 001 12 0370.jpg]&set=key[rotate],value[]&set=key[width],value[]&set=key[height],value[]&set=key[x],value[]&set=key[y],value[]&set=key[type],value[STILL_LIFE_FRONT]&set=key[hmver],value[1]&call=url[file:/product/large]',
  'model': ''},
 'metricCategoryID': 'LADIES_CARDIGANS_JUMPERS_CARDIGANS_W028',
 'name': 'Perforated-patterned Cardigan',
 'serial': '57154-A',
 'url': 'http://www.hm.com/us/product/57154?article=57154-A',
 'who': 'Women'}

### Serializing the items_info

In [27]:
import pickle
f = open('items_info.p', 'wb')
pickle.dump(items_info, f)
f.close()